In [1]:
import math

from uuid import uuid4

from vdom.svg import *

from sympy import symbols, sin, cos, pi, Point, Line, Point2D
from sympy.simplify import simplify

import regex

import lxml.etree as et 

from allofplos import starterdir, Corpus


from IPython.display import HTML


In [2]:
class ArchemedianSpiral:
    a, b, θ, ϕ = symbols('a,b,θ, ϕ')
    y1 = (a+b*θ)*sin(θ)
    x1 = (a+b*θ)*cos(θ)
    y2 = (a+b*ϕ)*sin(ϕ)
    x2 = (a+b*ϕ)*cos(ϕ)

    g = (b*sin(θ) + (a + b*θ)*cos(θ))/(b*cos(θ) - (a+b*θ)*sin(θ))
    h = (b*sin(ϕ) + (a + b*ϕ)*cos(ϕ))/(b*cos(ϕ) - (a+b*ϕ)*sin(ϕ))

    p1 = Point(x1,y1)
    p2 = Point(x2,y2)
    l1 = Line(p1, slope=g)
    l2 = Line(p2, slope=h)
    #self.int_point = self.l1.intersection(self.l2)[0]
    # hard code this so that it takes less time to create the class
    int_point = Point2D(
        (a**3*sin(θ) - a**3*sin(ϕ) + a**2*b*θ*sin(θ) - 2*a**2*b*θ*sin(ϕ) + 2*a**2*b*ϕ*sin(θ) 
         - a**2*b*ϕ*sin(ϕ) - a**2*b*cos(θ) + a**2*b*cos(ϕ) - a*b**2*θ**2*sin(ϕ) + 2*a*b**2*θ*ϕ*sin(θ) 
         - 2*a*b**2*θ*ϕ*sin(ϕ) + 2*a*b**2*θ*cos(ϕ) + a*b**2*ϕ**2*sin(θ) - 2*a*b**2*ϕ*cos(θ) - b**3*θ**2*ϕ*sin(ϕ) 
         + b**3*θ**2*cos(ϕ) + b**3*θ*ϕ**2*sin(θ) - b**3*ϕ**2*cos(θ))
        /(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - 
          b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ)), 
        (((a + b*θ)
          *(b*cos(θ) - (a + b*θ)*sin(θ))
          *(a**2*sin(θ - ϕ) + a*b*θ*sin(θ - ϕ) + a*b*ϕ*sin(θ - ϕ) + b**2*θ*ϕ*sin(θ - ϕ) + b**2*θ*cos(θ - ϕ) - b**2*ϕ*cos(θ - ϕ) + b**2*sin(θ - ϕ))
          *sin(θ)) 
         - ((b*sin(θ) + (a + b*θ)*cos(θ))
            *(-(b*sin(θ) + (a + b*θ)*cos(θ))
              *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
              *((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ)) 
              + ((b*cos(θ) - (a + b*θ)*sin(θ))
                *(b*cos(ϕ) - (a + b*ϕ)*sin(ϕ))
                *((a + b*θ)*sin(θ) - (a + b*ϕ)*sin(ϕ)))
              + ((a + b*θ)*cos(θ) - (a + b*ϕ)*cos(ϕ))
              *(a**2*sin(θ - ϕ) 
                + a*b*θ*sin(θ - ϕ) 
                + a*b*ϕ*sin(θ - ϕ) 
                + b**2*θ*ϕ*sin(θ - ϕ) 
                + b**2*θ*cos(θ - ϕ) 
                - b**2*ϕ*cos(θ - ϕ) 
                + b**2*sin(θ - ϕ))
             ))
        )/((b*cos(θ) - (a + b*θ)*sin(θ))
           *(a**2*sin(θ - ϕ) 
             + a*b*θ*sin(θ - ϕ) 
             + a*b*ϕ*sin(θ - ϕ) 
             + b**2*θ*ϕ*sin(θ - ϕ) 
             + b**2*θ*cos(θ - ϕ) 
             - b**2*ϕ*cos(θ - ϕ) 
             + b**2*sin(θ - ϕ))
          )
    )
    
    def __init__(self, a=0, b=1, num_cycles=3, angle=45, font_size=1, gen_quads=True):
        self.a = a
        self.b = b
        self.num_cycles = num_cycles
        self.angle = angle
        self.font_size = font_size
        self._quads = None
        if gen_quads:
            self.quads
        
    @property
    def center(self):
        return float(self.b*(self.num_cycles*pi*2)+14*self.font_size) + max(self.initial_point)
    
    @property
    def initial_point(self):
        """ Because quadratic curves are assuming that they are absolute we need to provide the starting point.
        """
        return list(map(float, self.p1.subs({"a":self.a, 
                                             "b":self.b, 
                                             "θ":0
                                            }).args))
    
    @property
    def quads(self):
        if self._quads is None: 
            self._quads = list(self.get_quad_control_points())
        return self._quads
    
    @property
    def curve_string(self):
        starting_point = map(lambda x: str(self.center + x), self.initial_point)
        starting_string = f'M{",".join(starting_point)}'        
        return f"{starting_string} {self.quad_string}"


    def quad_to_string(self, quad):
        ctrl_pt = ",".join(map(str, (q+self.center for q in quad[0])))
        final_pt = ",".join(map(str, (q+self.center for q in quad[1])))
        return f"Q{ctrl_pt} {final_pt}"
        
    @property
    def quad_string(self):       
        for quad in self.quads:
            return " ".join(self.quad_to_string(quad) for quad in self.quads)
            
        
    def get_intersection_points(self, num_cycles, angle_change, θ0=0):
        """
        num_cycles: 
            how many times does the spiral go around (this will be rounded up to give a complete segment)
        angle_change:
            how large is the change in angle for each quadratic segment
        """
        num_degs = num_cycles*360
        num_angles = math.floor(num_degs/angle_change)
        

        δθ = (pi/180)*angle_change
        for x in range(num_angles):
            loc_θ = θ0 + x*δθ
            yield {"int_point": self.int_point.subs({"a":self.a, 
                                                     "b":self.b, 
                                                     "θ":loc_θ, 
                                                     "ϕ":(loc_θ+δθ)}),
                   "loc_θ": loc_θ,
                   "δθ": δθ
                  }

    def get_quad_control_points(self, θ0=0):
        
        for x in self.get_intersection_points(self.num_cycles, self.angle, θ0=θ0):
            yield (tuple(map(float, x['int_point'].args)), 
                   tuple(map(float, self.p2.subs({"a":self.a, 
                                                  "b":self.b, 
                                                  "θ":x['loc_θ'], 
                                                  "ϕ":(x['loc_θ']+x['δθ'])
                                                 }).args))
                  )

In [81]:
def svgStyledSpiral(content = None, spiral=None, stroke="", **kwargs):
    """
    font_size: float 
        default is 1; interpreted as em (so 1 is notebook default)
    """

    if content is None:
        content = "The quick brown fox jumps over the lazy dog. "*10
    if spiral is None:
        #   a # how far out to start
        #   b # number of units between cycles
        #   num_cycles # minimum number of cycles to be covered
        #   angle # number of degrees each quadratic curve will approximate

        spiral = ArchemedianSpiral(a=kwargs["a"], b=kwargs["b"], 
                                   num_cycles=kwargs['num_cycles'], 
                                   angle=kwargs['angle'],
                                   font_size=kwargs['font_size'])
                     

    starting_point = map(lambda x: str(center + x), spiral.initial_point)
    font_size = spiral.font_size
    myid = f"{uuid4()}"
    side = spiral.center*2 + 1.5*font_size*14
    return svg(
        path(id=f"{myid}",
             fill="none", 
             stroke=stroke,
             d=spiral.curve_string
            ),
        text(
            textPath(content,
                     **{"xlink:href":f"#{myid}"}),
            style={"fontSize": f"{font_size}em",
                   "fontFamily": "Monospace",
                  }
        ),
        width=f"{side}",
        height=f"{side}"
    )


In [56]:
def svgAnimateSpiral(content = None, spiral=None, stroke="", **kwargs):
    """
    font_size: float 
        default is 1; interpreted as em (so 1 is notebook default)
    """

    if content is None:
        content = "The quick brown fox jumps over the lazy dog. "*10
    if spiral is None:
        #   a # how far out to start
        #   b # number of units between cycles
        #   num_cycles # minimum number of cycles to be covered
        #   angle # number of degrees each quadratic curve will approximate

        spiral = ArchemedianSpiral(a=kwargs["a"], 
                                   b=kwargs["b"], 
                                   num_cycles=kwargs['num_cycles'], 
                                   angle=kwargs['angle'],
                                   font_size=kwargs['font_size'])
    delay = kwargs.get("delay") or 0
    center = spiral.center
    font_size = spiral.font_size
    start_anim = animate(
                         attributeName="startOffset", 
                         dur=f"{spiral.num_cycles*1}s", 
                         repeatCount="indefinite",
                         begin=f"{delay}s",
                         **{"from": "0%",
                            "to": "100%"}
                     ) if kwargs.get("start_anim") else ""
    
    rotate_anim =   animateTransform(
                                    **{"attributeName":"transform",
                                       "attributeType":"XML",
                                       "type":"rotate",
                                       "from":f"360 {center} {center}",
                                       "to":f"0 {center} {center}",
                                       "dur":"1s",
                                       "repeatCount":"indefinite"}
                    ) if kwargs.get("rotate_anim") else ""
    starting_point = map(lambda x: str(center + x), spiral.initial_point)

    myid = f"{uuid4()}"
    side = spiral.center*2 + 1.5*font_size*14
    return svg(
        path(id=f"{myid}",
             fill="none", 
             stroke=stroke,
             d=spiral.curve_string
            ),
        text(
            textPath(start_anim,
                     *content,
                     **{"xlink:href":f"#{myid}"}
                    ),
            rotate_anim,
            style={"fontSize": f"{font_size}em",
                   "fontFamily": "Monospace",
                  }
        ),
        
        width=f"{side}",
        height=f"{side}"
    )


In [5]:
posix = regex.compile('[^\p{posix_punct}]')
uni = regex.compile('[^\p{p}]')
                     
def get_punct(article):
    return regex.sub(uni,'', str(article))

def body_punct(article):
    tags = ["/",
            "article",
            "body"]
    return get_punct(et.tostring(article.get_element_xpath(tags)[0], encoding='unicode'))
       
color_map = {"?!.": "red",
             "\"'()[]{}“”": "green",
             "—-,;:/\\&%*$@–": "blue"}

def color_wrap(char):
    color = "black"
    for key, val in color_map.items():
        if char in key:
            color = val
            
    return tspan(char, fill= f'{color}')

def style_punct(text, width=100):
    return [color_wrap(c) for c in text]


In [6]:
corpus = Corpus(starterdir)
article = corpus[0]

In [62]:
# for article in corpus.random_sample(1):
content = body_punct(article)
spiral = ArchemedianSpiral(a=60, b=2, num_cycles=9.25, angle=45, font_size=1)

In [82]:
display(svgStyledSpiral(content=style_punct(content), spiral=spiral))

<div><svg height="310.32741228718345" width="310.32741228718345"><path d="M149.66370614359172,144.66370614359172 Q150.23242825555673,146.08551142350422 149.90073284463102,147.68730491919 Q149.53625325316807,149.44741681608446 148.21090369478833,150.80763252674814 Q146.7681543149309,152.28833651206614 144.66370614359172,152.8052987971815 Q142.39151585170578,153.36346730021614 140.06931104119852,152.62143189098236 Q137.5795207057889,151.8258463413863 135.799080714084,149.78170002158322 Q133.90154552123326,147.6031151778175 133.38052083641213,144.66370614359172 Q132.82807070781394,141.5470075566174 133.98528134984977,138.49851471440363 Q135.20687511669064,135.280414466245 137.97491593880534,133.07838166773266 Q140.88576667282388,130.76274067838278 144.66370614359172,130.23892818282235 Q148.6232508194327,129.68993606847442 152.39969389957471,131.26458230349843 Q156.34599865095345,132.91005498431062 158.9697296658021,136.40411961201045 Q161.70431064068825,140.04580490743757 162.23007675795088,144.66370614359172 Q162.77677976626,149.46549997356323 160.78352903003633,153.9704902263696 Q158.71523020544012,158.64509833029015 154.49408900196792,161.69042871215345 Q150.12225466364978,164.8444762299199 144.66370614359172,165.3716694115407 Q139.0199140852476,165.91675363690044 133.78612573401895,163.50422807638768 Q128.38309743605197,161.01369101254954 124.91628452867869,156.0648853287628 Q121.34242568337802,150.96327372201597 120.81415022205297,144.66370614359172 Q120.27026812798564,138.1780329350007 123.10248516444446,132.21532940722403 Q126.01490185661768,126.08378049651986 131.69173063162575,122.19558548232735 Q137.52289884040164,118.20167971999473 144.66370614359172,117.67255756846318 Q151.99120197785996,117.12960210186844 158.6828792067543,120.38178611809313 Q165.54302944892208,123.71584874594112 169.85252585120742,130.12093430483085 Q174.26664577423765,136.68151891495262 174.79644737231007,144.66370614359172 Q175.33866740309537,152.8329940819321 171.66632521544165,160.2536755335492 Q167.91077691509605,167.84249111471163 160.7772743091475,172.57322489755876 Q153.48736722013376,177.4076816356913 144.66370614359172,177.93804002589985 Q135.65264231942746,178.47966251824104 127.50294042683936,174.38702426179296 Q119.18540859586946,170.2101047352504 114.03348834327338,162.3480706359424 Q108.77860232638541,154.3289067073215 108.24777960769379,144.66370614359172 Q107.70665202427523,134.81087498143253 112.21968897903915,125.93214410004447 Q116.81789527696723,116.88585516741978 125.40854532444615,111.31278929692205 Q134.15691860833334,105.63740264965503 144.66370614359172,105.106186954104 Q155.35830035637994,104.56547588668042 164.96606451393387,109.49898993268783 Q174.74114367842884,114.5184189661278 180.73532203661273,123.83774899765126 Q186.8312654851408,133.31529628697626 187.36281798666926,144.66370614359172 Q187.90317374117464,156.20006154193 182.54912140084696,166.5368608407288 Q177.10851930322337,177.04075739851186 167.06045961632708,183.45602108296407 Q156.85376528150528,189.97256632103398 144.66370614359172,190.50441064025904 Q132.28559003132875,191.04445976246058 121.21975511965977,185.26982044719827 Q109.98701866660484,179.40808456756838 103.15069215786806,168.63125594312197 Q96.21350855449334,157.69543555269078 95.68140899333463,144.66370614359172 Q95.14162715687235,131.44382898863847 101.33689279363385,119.64895879286487 Q107.61973086739007,107.68736361907722 119.12536001726657,100.42999311151674 Q130.79028982785536,93.0721410134384 144.66370614359172,92.53981633974482 Q158.72534512078278,92.00026953960739 171.24924982111347,98.61619374728252 Q183.93971960883644,105.32010824394501 191.61811822201804,117.55456369047168 Q199.39666392346868,129.94858953067703 199.92918860102841,144.66370614359172 Q200.46852712329186,159.5671079639768 193.43191758625227,172.82004614790836 Q186.30694796109725,186.2394040798103 173.34364492350667,194.33881726836935 Q160.22053398025287,202.53807771460737 144.66370614359172,203.07078125461823 Q128.91854

In [67]:
spiral = ArchemedianSpiral(a=5, b=2, num_cycles=10, angle=30, font_size=1)

In [75]:
def pretty_corpus(corpus):

    rep = []
    for idx, article in enumerate(corpus.random_sample(3)):
        svg = svgAnimateSpiral(content=style_punct(body_punct(article)), 
                               spiral=spiral,
                               delay=idx/2,
                               start_anim=True,
                               rotate_anim=True
                              )
        rep += svg.to_html()
        
    return "".join(rep)

In [76]:
html = get_ipython().display_formatter.formatters['text/html']
html.for_type(Corpus, pretty_corpus)

html = get_ipython().display_formatter.formatters['text/html']



<function __main__.pretty_corpus(corpus)>

In [77]:
corpus

Corpus location: /Users/mpacer/jupyter/allofplos/allofplos/starter_corpus
Number of files: 122